In [3]:
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
import torch
import torch.nn as nn

In [5]:
binarized = torch.load('./data/binarized.pt')

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Сверточная сеть

In [7]:
class ConvNet(nn.Module):
    def __init__(self):

        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
             nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
             nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2))
        self.bn1 = nn.BatchNorm1d(7 * 7 * 64)
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.relu1 = nn.ReLU()
        
        self.drop_out2 = nn.Dropout(0.1)
        self.bn2 = nn.BatchNorm1d(1000)
        self.fc2 = nn.Linear(1000, 500)
        self.relu2 = nn.ReLU()
        
        self.drop_out3 = nn.Dropout(0.2)
        self.bn3 = nn.BatchNorm1d(500)
        self.fc3 = nn.Linear(500, 10)
      
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.bn1(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.drop_out2(out)
        out = self.bn2(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.drop_out3(out)
        out = self.bn3(out)
        out = self.fc3(out)
        return out

In [9]:
model = torch.load('./models/mnist-cnn-model.pth')
model.eval()

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (bn1): BatchNorm1d(3136, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3136, out_features=1000, bias=True)
  (relu1): ReLU()
  (drop_out2): Dropout(p=0.1, inplace=False)
  (bn2): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (relu2): ReLU()
  (drop_out3): Dropout(p=0.2, inplace=False)
  (bn3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=500, out_features=10, bias=True)


### Предсказания для бинаризованных изображений сверточной сетью

In [27]:
model = model.to(device)

In [28]:
criterion = nn.CrossEntropyLoss()

In [29]:
with torch.no_grad():
    images = binarized.tensors[0].float().to(device)
    labels = binarized.tensors[1].to(device)
    output = model(images.unsqueeze(1))  
    loss = criterion(output, labels)
    print('Final validation Loss {}'.format(loss))

Final validation Loss 5.768934726715088


In [30]:
valid_acc = torch.sum(torch.argmax(nn.functional.softmax(output, dim=1), dim=1) == labels) / 10000
print('Final validation Accuracy {} %'.format(valid_acc*100))

Final validation Accuracy 11.350000381469727 %


### Полносвязная сеть

In [11]:
class MNISTNet(torch.nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.fc1 = torch.nn.Linear(28*28, 1024)
        self.ac1 = torch.nn.ReLU()
        self.drop_out1 = nn.Dropout(0.2)
        self.bn1 = nn.BatchNorm1d(1024)
        
        self.fc2 = torch.nn.Linear(1024, 512)
        self.ac2 = torch.nn.ReLU()
        self.drop_out2 = nn.Dropout(0.3)
        self.bn2 = nn.BatchNorm1d(512)
        
        self.fc3 = torch.nn.Linear(512, 128)
        self.ac3 = torch.nn.ReLU()
        self.drop_out3 = nn.Dropout(0.4)
        self.bn3 = nn.BatchNorm1d(128)
        
        self.fc4 = torch.nn.Linear(128, 10)
        
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.drop_out1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.ac2(x)
        x = self.drop_out2(x)
        x = self.bn2(x)
        x = self.fc3(x)
        x = self.ac3(x)
        x = self.drop_out3(x)
        x = self.bn3(x)
        x = self.fc4(x)
        return x

In [12]:
model_fcnn = torch.load('./models/mnist-fcnn-model.pth')
model_fcnn.eval()

MNISTNet(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (ac1): ReLU()
  (drop_out1): Dropout(p=0.2, inplace=False)
  (bn1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (ac2): ReLU()
  (drop_out2): Dropout(p=0.3, inplace=False)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=512, out_features=128, bias=True)
  (ac3): ReLU()
  (drop_out3): Dropout(p=0.4, inplace=False)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
)

### Предсказания полносвязной сетью для бинаризованных изображений

In [33]:
with torch.no_grad():
    images = binarized.tensors[0].reshape([-1,28*28]).float().to(device)
    labels = binarized.tensors[1].to(device)
    output = model_fcnn(images)  
    loss = criterion(output, labels)
    print('Final validation Loss {}'.format(loss))

Final validation Loss 2.4645705223083496


In [34]:
valid_acc = torch.sum(torch.argmax(nn.functional.softmax(output, dim=1), dim=1) == labels) / 10000
print('Final validation Accuracy {} %'.format(valid_acc*100))

Final validation Accuracy 8.920000076293945 %
